Weil pairings over $\mathbf F_p$
--------------

The aim of this notebook is to compute Weil pairings on the $p$-power torsion of an ordinary elliptic curve over $\mathbf F_p$, following Belding, Juliana V. “A Weil Pairing on the P-Torsion of Ordinary Elliptic Curves over K[ϵ].” Journal of Number Theory 128, no. 6 (June 2008): 1874–88. https://doi.org/10.1016/j.jnt.2008.02.002.

The difficulty being that for an elliptic curve over $\mathbf F_p$ the $p^n$-torsion is only $\mathbf Z/p^n$ geometrically, as the group scheme splits into an etale part $\mathbf Z/p^n$ which has $p^n$ points for any field, and a connected part $ \mu_{p^n}$ which does not have geometric points over  a characteristic field.

To see the missing points the dual numbers $\mathbf F_p[\epsilon]$ are used.

Let us fix an elliptic curve and a $p$ for now.

In [1]:
from sage.rings.padics.precision_error import PrecisionError
p = 5
K = GF(p)
E = EllipticCurve(K, [3,0]); show(E)

Elliptic Curve defined by y^2 = x^3 + 3*x over Finite Field of size 5

In [2]:
E.abelian_group()

Additive abelian group isomorphic to Z/10 embedded in Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 3*x over Finite Field of size 5

In [3]:
E.points()

[(0 : 0 : 1), (0 : 1 : 0), (1 : 2 : 1), (1 : 3 : 1), (2 : 2 : 1), (2 : 3 : 1), (3 : 1 : 1), (3 : 4 : 1), (4 : 1 : 1), (4 : 4 : 1)]

Now we lift to the dual numbers using the natural inclusion
$$K \hookrightarrow K[\epsilon]$$

In [4]:
R.<x> = PolynomialRing(K)
Ke.<e> = R.quo(x^2)

In [5]:
Ee = E.change_ring(Ke); show(Ee)

Elliptic Curve defined by y^2 = x^3 + 3*x over Univariate Quotient Polynomial Ring in e over Finite Field of size 5 with modulus x^2

Now we have various extra points, such as those at infinity with tangent information.

In [6]:
Ee(e,1,0), Ee(-e, 1, 0)

((e : 1 : 0), (4*e : 1 : 0))

Unfortunately Sage does not like to do things like add these points.

In [7]:
Ee(-e, 1, 0) + Ee(1, 0) # errors

TypeError: Coordinates [1, 0, 1] do not define a point on Elliptic Curve defined by y^2 = x^3 + 3*x over Univariate Quotient Polynomial Ring in e over Finite Field of size 5 with modulus x^2

Thus we must unfortunately reimplement elliptic curve addition in a way that works over more general rings.

We'll use the equations given by Washington in  Elliptic Curves: Number Theory and Cryptography, Second Edition p. 67

In [16]:
def normalize(self):
    """
    Attempt to normalize an elliptic curve point by dividing by the y-coordinate
    """
    try:
        u = self[2]^(-1)
    except (ZeroDivisionError, PrecisionError):
        try:
            u = self[1]^(-1)
        except (ZeroDivisionError, PrecisionError):
            u = self[0]^(-1)
    x,y,z = self
    return self.scheme()(u*x,u*y,u*z)
def neg(self):
    E = self.scheme()
    assert E.short_weierstrass_model() == E
    return normalize(E(self[0], -self[1], self[2]))
def add(self, other):
    """
    Add two points on an elliptic curve over a ring, using the formulae in Washington.
    
    Currently this is slow and stupid.
    """
    E = self.scheme()
    assert E.short_weierstrass_model() == E
    assert E == other.scheme()
    _, _, _, A, B = E.a_invariants()
    x_1,y_1,z_1 = self
    x_2,y_2,z_2 = other
    
    X1 = (x_1*y_2-x_2*y_1)*(y_1*z_2+y_2*z_1)+(x_1*z_2-x_2*z_1)*y_1*y_2\
    -A*(x_1*z_2+x_2*z_1)*(x_1*z_2-x_2*z_1)-3*B*(x_1*z_2-x_2*z_1)*z_1*z_2
    Y1 = -3*x_1*x_2*(x_1*y_2-x_2*y_1)-y_1*y_2*(y_1*z_2-y_2*z_1)-A*(x_1*y_2-x_2*y_1)*z_1*z_2\
    +A*(x_1*z_2+x_2*z_1)*(y_1*z_2-y_2*z_1)+3*B*(y_1*z_2-y_2*z_1)*z_1*z_2
    Z1 = 3*x_1*x_2*(x_1*z_2-x_2*z_1)-(y_1*z_2+y_2*z_1)*(y_1*z_2-y_2*z_1)\
    +A*(x_1*z_2-x_2*z_1)*z_1*z_2

    X2 = y_1*y_2*(x_1*y_2+x_2*y_1)-A*x_1*x_2*(y_1*z_2+y_2*z_1)\
    -A*(x_1*y_2+x_2*y_1)*(x_1*z_2+x_2*z_1)-3*B*(x_1*y_2+x_2*y_1)*z_1*z_2\
    -3*B*(x_1*z_2+x_2*z_1)*(y_1*z_2+y_2*z_1)+A^2*(y_1*z_2+y_2*z_1)*z_1*z_2
    Y2 = y_1^2*y_2^2+3*A*x_1^2*x_2^2+9*B*x_1*x_2*(x_1*z_2+x_2*z_1)\
    -A^2*x_1*z_2*(x_1*z_2+2*x_2*z_1)-A^2*x_2*z_1*(2*x_1*z_2+x_2*z_1)\
    -3*A*B*z_1*z_2*(x_1*z_2+x_2*z_1)-(A^3+9*B^2)*z_1^2*z_2^2
    Z2 = 3*x_1*x_2*(x_1*y_2+x_2*y_1)+y_1*y_2*(y_1*z_2+y_2*z_1)+A*(x_1*y_2+x_2*y_1)*z_1*z_2\
    +A*(x_1*z_2+x_2*z_1)*(y_1*z_2+y_2*z_1)+3*B*(y_1*z_2+y_2*z_1)*z_1*z_2

    X3 = (x_1*y_2+x_2*y_1)*(x_1*y_2-x_2*y_1)+A*x_1*x_2*(x_1*z_2-x_2*z_1)\
    +3*B*(x_1*z_2+x_2*z_1)*(x_1*z_2-x_2*z_1)-A^2*(x_1*z_2-x_2*z_1)*z_1*z_2
    Y3 = (x_1*y_2-x_2*y_1)*y_1*y_2-3*A*x_1*x_2*(y_1*z_2-y_2*z_1)\
    +A*(x_1*y_2+x_2*y_1)*(x_1*z_2-x_2*z_1)+3*B*(x_1*y_2-x_2*y_1)*z_1*z_2\
    -3*B*(x_1*z_2+x_2*z_1)*(y_1*z_2-y_2*z_1)+A^2*(y_1*z_2-y_2*z_1)*z_1*z_2
    Z3 = -(x_1*y_2+x_2*y_1)*(y_1*z_2-y_2*z_1)-(x_1*z_2-x_2*z_1)*y_1*y_2\
    -A*(x_1*z_2+x_2*z_1)*(x_1*z_2-x_2*z_1)-3*B*(x_1*z_2-x_2*z_1)*z_1*z_2

    M = Matrix(3,3,[X1, Y1, Z1, X2, Y2, Z2, X3, Y3, Z3])
    assert M.adjugate() == 0
    R = E.base_ring()
    I = R.ideal(0)
    L = MatrixSpace(E.base_ring(), 1, 3)
    L2 = MatrixSpace(E.base_ring(), 3, 1)
    # TODO this is a ridiculous way of finding a primitive linear combination of the rows
    # but it works for now.
    verbose(M)
    #M = M._echelon_in_place_classical()
    while I != 1:
        verbose(M)
        o = sum(M.rows())
        l2 = L2.random_element()
        try:
            verbose(("asd",o))
            I = R.ideal(o)
            verbose(I)
        except Exception as e:
            verbose(e)
            I = M*l2
    
    verbose(o[0],level=2)
    return normalize(E(list(o[0])))

def normalize_affine(self):
    """
    Attempt to normalize an elliptic curve point by dividing by the z-coordinate
    """
    u = self[2]^(-1)
    x,y,z = self
    return self.scheme()(u*x,u*y,u*z)

def normalize_near_inf(self):
    """
    Attempt to normalize an elliptic curve point by dividing by the y-coordinate
    """
    u = self[1]^(-1)
    x,y,z = self
    return self.scheme()(u*x,u*y,u*z)



In [18]:
R.ideal()

NameError: name 'o' is not defined

Let's test that this works

In [17]:
Oe = Ee(e,1,0)
normalize(add(Oe,Oe))

KeyboardInterrupt: 

In [10]:
print(normalize_near_inf(add(Oe,Oe)))
print(normalize_near_inf(add(add(Oe,Oe), Oe)))
print(normalize_near_inf(add(Oe,neg(Oe))))
assert normalize_near_inf(add(Oe,neg(Oe))) == Ee(0)

ZeroDivisionError: element 2*e of quotient polynomial ring not invertible

In [11]:
Q = E(3,1)
5*Q

(0 : 0 : 1)

In [12]:
P = Ee(4,4,1)
Q = Ee(3,1,1)
normalize(add(Oe,Q))

(3*e + 3 : 1 : 1)

In [13]:
T = Ee(0, 0)
add(T, T)

(0 : 1 : 0)

Now given a point of $E(K[\epsilon])$ we can specialise to $\epsilon = 0$, so we in fact have a split short exact sequence
$$ 0 \to \Theta \to E(K[\epsilon]) \to E(K)\to0$$
where the kernel $\Theta$ is simply the points specialising to infinity. This is the tangent space at infinity.

We now define the section $E(K[\epsilon]) \to \Theta$

In [14]:
def specialization(self):
    x,y,z = self
    x = x.parent()(x.lift().subs(0))
    y = y.parent()(y.lift().subs(0))
    z = z.parent()(z.lift().subs(0))
    return self.scheme()(x, y, z)
def tangent_part(self, check=True):
    x,y,z=self
    return add(self, neg(specialization(self)))
def tangent_part2(self, check=True):
    x,y,z=self
    print(x[0],x[1])
    def1 = -x[1]/(2*y[0]) if y[0] != 0 else -y[1]/(3*x[0]^2 + self.scheme().a6())
    def2 = tangent_part(self)
    print(def1,def2)
    assert def2[0] == def1*e

In [15]:
P
normalize_near_inf(tangent_part(P))

(0 : 1 : 0)

Now we define the Weil pairing on $p$-torsion, starting with $E[p]\times \Theta$.

In [16]:
R = add(P,Q)
print(R)
normalize_near_inf(tangent_part(R))

(2 : 2 : 1)


(0 : 1 : 0)

In [17]:
S = E(1,0)

TypeError: Coordinates [1, 0, 1] do not define a point on Elliptic Curve defined by y^2 = x^3 + 3*x over Finite Field of size 5

In [19]:
def line(self, R, Q):
    r"""
    Computes the value at `Q` of a straight line through points
    self and `R`.

    INPUT:

    - ``R, Q`` -- points on self.curve() with ``Q`` nonzero.

    OUTPUT:

    An element of the base field self.curve().base_field().
    A ValueError is raised if ``Q`` is zero.

    EXAMPLES::

        sage: F.<a>=GF(2^5)
        sage: E=EllipticCurve(F,[0,0,1,1,1])
        sage: P = E(a^4 + 1, a^3)
        sage: Q = E(a^4, a^4 + a^3)
        sage: O = E(0)
        sage: P._line_(P,-2*P) == 0
        True
        sage: P._line_(Q,-(P+Q)) == 0
        True
        sage: O._line_(O,Q) == F(1)
        True
        sage: P._line_(O,Q) == a^4 - a^4 + 1
        True
        sage: P._line_(13*P,Q) == a^4
        True
        sage: P._line_(P,Q) == a^4 + a^3 + a^2 + 1
        True

    See :trac:`7116`::

        sage: P._line_ (Q,O)
        Traceback (most recent call last):
        ...
        ValueError: Q must be nonzero.

    .. NOTE::

        This function is used in _miller_ algorithm.

    AUTHOR:

    - David Hansen (2009-01-25)
    """
    if Q.is_zero():
        raise ValueError("Q must be nonzero.")

    if self.is_zero() or R.is_zero():
        if self == R:
            return self.scheme().base_ring().one()
        if self.is_zero():
            return Q[0] - R[0]
        if R.is_zero():
            return Q[0] - self[0]
    elif self != R:
        if self[0] == R[0]:
            return Q[0] - self[0]
        else:
            l = (R[1] - self[1])/(R[0] - self[0])
            return Q[1] - self[1] - l * (Q[0] - self[0])
    else:
        a1, a2, a3, a4, a6 = self.scheme().a_invariants()
        numerator = (3*self[0]**2 + 2*a2*self[0] + a4 - a1*self[1])
        denominator = (2*self[1] + a1*self[0] + a3)
        if denominator == 0:
            return Q[0] - self[0]
        else:
            l = numerator/denominator
            return Q[1] - self[1] - l * (Q[0] - self[0])
        
def miller(self, Q, n):
    r"""
    Return the value at `Q` of the rational function `f_{n,P}`, where the
    divisor of `f_{n,P}` is `n[P]-[nP]-(n-1)[O]`.

    INPUT:

    - ``Q`` -- a nonzero point on self.curve().

    - ``n`` -- an nonzero integer. If `n<0` then return `Q`
               evaluated at `1/(v_{nP}*f_{n,P})` (used in the ate pairing).

    OUTPUT:

    An element in the base field self.curve().base_field()
    A ValueError is raised if `Q` is zero.

    EXAMPLES::

        sage: F.<a>=GF(2^5)
        sage: E=EllipticCurve(F,[0,0,1,1,1])
        sage: P = E(a^4 + 1, a^3)
        sage: Fx.<b>=GF(2^(4*5))
        sage: Ex=EllipticCurve(Fx,[0,0,1,1,1])
        sage: phi=Hom(F,Fx)(F.gen().minpoly().roots(Fx)[0][0])
        sage: Px=Ex(phi(P.xy()[0]),phi(P.xy()[1]))
        sage: Qx = Ex(b^19 + b^18 + b^16 + b^12 + b^10 + b^9 + b^8 + b^5 + b^3 + 1, b^18 + b^13 + b^10 + b^8 + b^5 + b^4 + b^3 + b)
        sage: Px._miller_(Qx,41) == b^17 + b^13 + b^12 + b^9 + b^8 + b^6 + b^4 + 1
        True
        sage: Qx._miller_(Px,41) == b^13 + b^10 + b^8 + b^7 + b^6 + b^5
        True
        sage: P._miller_(E(0),41)
        Traceback (most recent call last):
        ...
        ValueError: Q must be nonzero.

    An example of even order::

        sage: F.<a> = GF(19^4)
        sage: E = EllipticCurve(F,[-1,0])
        sage: P = E(15*a^3 + 17*a^2 + 14*a + 13,16*a^3 + 7*a^2 + a + 18)
        sage: Q = E(10*a^3 + 16*a^2 + 4*a + 2, 6*a^3 + 4*a^2 + 3*a + 2)
        sage: x=P.weil_pairing(Q,360)
        sage: x^360 == F(1)
        True

    You can use the _miller_ function on linearly dependent
    points, but with the risk of a dividing with zero::

        sage: Px._miller_(2*Px,41)
        Traceback (most recent call last):
        ...
        ZeroDivisionError: division by zero in finite field

    A small example of embedding degree 6::

        sage: q = 401; F = GF(q); a = 146; b = 400; k = 6
        sage: E = EllipticCurve([F(a), F(b)])
        sage: R.<x> = F[]; K.<a> = GF(q^k, modulus=x^6 + 4*x^4 + 115*x^3 + 81*x^2 + 51*x + 3)
        sage: EK = E.base_extend(K)
        sage: P = E([F(338), F(227)])
        sage: Q_x = 333*a^5 + 391*a^4 + 160*a^3 + 335*a^2 + 71*a + 93
        sage: Q_y = 343*a^5 + 273*a^4 + 26*a^3 + 342*a^2 + 340*a + 210
        sage: Q = EK([Q_x, Q_y])
        sage: P._miller_(Q, 127)
        371*a^5 + 39*a^4 + 355*a^3 + 233*a^2 + 20*a + 275

    A series of small examples and small torsions.  We start with
    `n=1`, which is trivial: the function is the constant
    1::

        sage: E = EllipticCurve([GF(7)(0), 2])
        sage: P = E(5, 1); Q = E(0, 3); I = E(0)
        sage: I._miller_(P, 1)
        1
        sage: I._miller_(Q, 1)
        1

    A two-torsion example. In this case `f_{n,P}(Q) = x_Q - x_P`::

        sage: E = EllipticCurve([GF(7)(-1), 0])
        sage: P = E(0,0); Q = E(1, 0);
        sage: P._miller_(P, 2)
        0
        sage: Q._miller_(Q, 2)
        0
        sage: P._miller_(Q, 2)
        1
        sage: Q._miller_(P, 2)
        6

    A three-torsion example::

        sage: E = EllipticCurve([GF(7)(0), 2])
        sage: P = E(5, 1); Q = E(0, 3);
        sage: P._miller_(Q, 3)
        4

    A 4-torsion example::

        sage: E = EllipticCurve([GF(7)(-1), 0])
        sage: P = E(5, 1); Q = E(4, 2)
        sage: P._miller_(Q, 4)
        3

    A 5-torsion example::

        sage: E = EllipticCurve([GF(7)(-1), 4])
        sage: P = E(4, 1); Q = E(6, 5)
        sage: P._miller_(Q, 5)
        1

    A 6-torsion example::

        sage: E = EllipticCurve([GF(7)(3), 1])
        sage: P = E(5, 1); Q = E(3, 3)
        sage: P._miller_(Q, 6)
        5

    An example which is part of an ate pairing calculation. The trace of
    the curve is negative, so it should exercise the `n<0` case in the
    code::

        sage: p = 2017; A = 1; B = 30; r = 29; t = -70; k = 7;
        sage: F = GF(p); R.<x> = F[]
        sage: E = EllipticCurve([F(A), F(B)]); P = E(369, 716)
        sage: K.<a> = GF(p^k, modulus=x^k+2); EK = E.base_extend(K)
        sage: Qx = 1226*a^6 + 1778*a^5 + 660*a^4 + 1791*a^3 + 1750*a^2 + 867*a + 770
        sage: Qy = 1764*a^6 + 198*a^5 + 1206*a^4 + 406*a^3 + 1200*a^2 + 273*a + 1712
        sage: Q = EK(Qx, Qy)
        sage: Q._miller_(P, t-1)
        1311*a^6 + 1362*a^5 + 1177*a^4 + 807*a^3 + 1331*a^2 + 1530*a + 1931

    ALGORITHM:

        Double-and-add. See also [Mil2004]_.

    AUTHORS:

    - David Hansen (2009-01-25)
    - Mariah Lenox (2011-03-07) -- Added more doctests and support for
      negative n.
    """
    if Q.is_zero():
        raise ValueError("Q must be nonzero.")
    if n.is_zero():
        raise ValueError("n must be nonzero.")
    n_is_negative = False
    if n < 0:
        n = n.abs()
        n_is_negative = True

    one = self.scheme().base_ring().one()
    t = one
    V = self
    nbin = n.bits()
    i = n.nbits() - 2
    while i > -1:
        S = add(V,V)
        verbose(S,level=2)
        ell = line(V,V, Q)
        vee = line(S,neg(S), Q)
        verbose(ell,level=2)
        verbose(vee,level=2)
        t = (t**2)*(ell/vee)
        V = S
        if nbin[i] == 1:
            S = add(V,self)
            ell = line(V,self, Q)
            vee = line(S, neg(S), Q)
            verbose(ell,level=2)
            verbose(vee,level=2)
            t = t*(ell/vee)
            V = S
        i = i-1
    if n_is_negative:
        vee = V._line_(-V, Q)
        t = 1/(t*vee)
    return t

In [20]:
def mul(n, P):
    if n < 0:
        return mul(-n, neg(P))
    if n == 1:
        return P
    return add(P, mul(n-1, P))
def vert(P, Q):
    return line(P, neg(P), Q)
    #return line(P, P.scheme()(0), Q)

Let's first do an example by hand
$$5 =  4 + 1  = (2 + 2) + 1 = ((1 + 1) + (1 + 1)) + 1$$
so
$$\begin{align}f_5 &= f_1 f_4 h_{1,4}\\\
&=f_1f_2^2 h_{1,4} h_{2,2}\\
&=f_1^5 h_{1,4} h_{2,2} h_{1,1}^2
\end{align}$$ 
now we have
$$e (Q, P) = \prod_C \frac{h_{i,j}(R)}{h_{i,j}(P + R)}$$
for $C$ an addition chain decomposition for $p = 5$.
That is
$$ e(Q,P) = \frac{ h_{1,4} (R) h_{2,2} (R)h_{1,1}^2 (R)}{h_{1,4} (R + P) h_{2,2} (R + P)h_{1,1}^2 (R + P)}$$
starting from the left $h_{1,4} (R) = v_1(R-T)$ where $v_1$ is a vertical line through $Q$.

In [21]:
Oe = Ee(e,1,0)
P = mul(2,Ee(4,4,1))
R = mul(2,P)
T = Ee(0,0)

In [22]:
h14n = vert(Q, add(R, neg(T))); h14n

4

In [23]:
h14d = vert(Q, add(add(R, P), neg(T))); h14d

4

Next $h_{2,2}(R) = \ell_{2,2}/v_{2} (R-T)$

In [24]:
h2n = line(mul(2, Q), mul(2, Q), add(R, neg(T)))/ vert(mul(4, Q), add(R, neg(T))); h2n

2

In [25]:
assert add(add(R, neg(T)),P) == add(add(R, P), neg(T))

In [26]:
h2d = line(mul(2, Q), mul(2, Q), add(add(R, neg(T)),P))/ vert(mul(4, Q),add(add(R, neg(T)),P)); h2d

3

In [27]:
h1n = line( Q,  Q, add(R, neg(T)))/ vert(mul(2,Q), add(R, neg(T))); h1n

2

In [28]:
h1d = line( Q, Q, add(add(R, neg(T)),P))/ vert(mul(2,Q), add(add(R, neg(T)),P)); h1d

4

In [29]:
eQP = (h14n*h2n*h1n^2)/(h14d*h2d*h1d^2); eQP

1

In [30]:
def hij(i, j, n, T, P, ev):
    evT = add(ev, neg(T))
    assert evT == add(neg(T), ev)
    if i + j == n:
        return vert(mul(i, P), evT)
    return line(mul(i, P), mul(j, P), evT) /vert(mul(i + j, P), evT)
def binary_addition_chain(n):
    # list of triples (i, j, i+j) such that i, j both appear as i+j for some earlier
    l = [(0,0,0)]
    for i in reversed(n.bits()):
        l.append((l[-1][2], l[-1][2], 2*l[-1][2]))
        if i:
            l.append((1, 2*l[-2][2], 2*l[-2][2]+1))
    return (l[3:])
list(binary_addition_chain(16))
def check_addition_chain(C):
    rhs = [c[2] for c in C]
    if sorted(rhs) != rhs:
        verbose("not sorted")
        return False
    if len(set(rhs)) != len(rhs):
        verbose("not unique")
        return False
    for c in C:
        if c[0] not in rhs and c[0] != 1:
            verbose("not member left %s"%c[0])
            return False
        if c[1] not in rhs and c[1] != 1:
            verbose("not member right")
            return False
    return True
from collections import defaultdict
def addition_chain_to_mult(C):
    r"""
    Convert an addition chain into an addition chain with multiplicities.
    """
    if not check_addition_chain(C):
        raise ValueError
    nneeded = defaultdict(ZZ)
    n = C[-1][2]
    nneeded[n] = 1
    for c in sorted(C, key=lambda C: -C[2]):
        verbose(c)
        nneeded[c[0]] += nneeded[c[2]]
        nneeded[c[1]] += nneeded[c[2]]
    return {c : nneeded[c[-1]] for c in C}

set_verbose(1)
addition_chain_to_mult(binary_addition_chain(9))

verbose 1 (addition_chain_to_mult) (1, 8, 9)
verbose 1 (addition_chain_to_mult) (4, 4, 8)
verbose 1 (addition_chain_to_mult) (2, 2, 4)
verbose 1 (addition_chain_to_mult) (1, 1, 2)


{(1, 1, 2): 4, (2, 2, 4): 2, (4, 4, 8): 1, (1, 8, 9): 1}

In [31]:
def special_weil(P, Q, pn):
    EP = P.scheme()
    Pt = tangent_part(P)
    p, n = pn.is_prime_power(get_data=True)
    assert p %2 == 1
    assert mul(pn,P) == EP(0)
    assert mul(pn,Q) == EP(0)
    try:
        tx = EP.two_division_polynomial().roots(multiplicities=False)[0]
    except IndexError:
        raise ValueError("No root for two division polynomial found, enlarge your base ring and retry")
    T = EP(tx,0)
    verbose(T)
    R = mul(1, P)
    pro = 1
    am = addition_chain_to_mult(binary_addition_chain(pn))
    for (i,j,ij) in am:
        verbose((i,j,am[(i,j,ij)]))
        cu = (hij(i, j, pn, T, P, R)/hij(i, j, pn, T, P, add(Q, R)))^am[(i,j,ij)]
        verbose(cu)
        pro *= cu
        verbose(pro)
    #if T != 0:
    #    return special_weil(add(P, neg(T)), Q)/special_weil(Q, T)
    return pro
set_verbose(1)
special_weil(mul(1,P),mul(1,Oe), 5)

verbose 1 (special_weil) (0 : 0 : 1)
verbose 1 (addition_chain_to_mult) (1, 4, 5)
verbose 1 (addition_chain_to_mult) (2, 2, 4)
verbose 1 (addition_chain_to_mult) (1, 1, 2)
verbose 1 (special_weil) (1, 1, 2)
verbose 1 (special_weil) 1
verbose 1 (special_weil) 1
verbose 1 (special_weil) (2, 2, 1)
verbose 1 (special_weil) 2*e + 1
verbose 1 (special_weil) 2*e + 1
verbose 1 (special_weil) (1, 4, 1)
verbose 1 (special_weil) 4*e + 1
verbose 1 (special_weil) e + 1


e + 1

In [95]:
R = Zp(5,3, "capped-abs")
E5= EllipticCurve(R, [3,0])
P = E5.lift_x(4+3*5 + 25)
print(P)
#(4 + O(5^2) : 1 + O(5^2) : 1 + O(5^2))
Q = E5.lift_x(4 + 5);print(Q)
#(4 + 5 + O(5^2) : 1 + 3*5 + O(5^2) : 1 + O(5^2))
Oe = E5(5, 1,0); print(Oe)
#(5 + O(5^2) : 1 + O(5^2) : O(5^2))
print(add(Q, neg(P)))
#verbose 1 (add) [O(5^2) O(5^2) O(5^2)]
#[O(5^3) O(5^2) O(5^3)][O(5^2) O(5^2) O(5^2)](3*5 + O(5^2), 4 + 2*5 + O(5^2), O(5^2))
#(2*5 + O(5^2) : 1 + O(5^2) : O(5^2))
#print(5*E5.teichmuller(P))

(4 + 3*5 + 5^2 + O(5^3) : 1 + 4*5 + O(5^3) : 1 + O(5^3))
(4 + 5 + O(5^3) : 1 + 3*5 + O(5^3) : 1 + O(5^3))
(5 + O(5^3) : 1 + O(5^3) : O(5^3))
verbose 1 (add) [      3*5 + 5^2 + O(5^3)               3 + O(5^3)                   O(5^3)]
[    4*5 + 3*5^2 + O(5^3)       4 + 2*5^2 + O(5^3)                   O(5^3)]
[    3*5 + 3*5^2 + O(5^3) 3 + 2*5 + 2*5^2 + O(5^3)                   O(5^3)]
verbose 1 (add) [      2*5^2 + O(5^3) 2*5 + 4*5^2 + O(5^3)               O(5^3)]
verbose 1 (add) [    3*5 + O(5^3) 3 + 4*5 + O(5^3)           O(5^3)]
verbose 1 (add) [3*5 + 2*5^2 + O(5^3) 3 + 5 + 5^2 + O(5^3)               O(5^3)]
verbose 1 (add) [    3*5 + O(5^3) 3 + 4*5 + O(5^3)           O(5^3)]
verbose 1 (add) [      4*5 + 5^2 + O(5^3) 4 + 3*5 + 3*5^2 + O(5^3)                   O(5^3)]
verbose 1 (add) [    3*5 + O(5^3) 3 + 4*5 + O(5^3)           O(5^3)]
verbose 1 (add) [      4*5 + 5^2 + O(5^3) 4 + 3*5 + 4*5^2 + O(5^3)                   O(5^3)]
verbose 1 (add) [            3*5 + O(5^3) 3 + 4*5 + 4*5^2

verbose 1 (add) [            3*5 + O(5^3) 3 + 4*5 + 3*5^2 + O(5^3)                   O(5^3)]
verbose 1 (add) [2*5 + 4*5^2 + O(5^3)     2 + 5^2 + O(5^3)               O(5^3)]
verbose 1 (add) [      5 + 4*5^2 + O(5^3) 1 + 2*5 + 3*5^2 + O(5^3)                   O(5^3)]
verbose 1 (add) [4*5 + 4*5^2 + O(5^3) 4 + 5 + 5^2 + O(5^3)               O(5^3)]
verbose 1 (add) [3*5 + 5^2 + O(5^3) 3 + 4*5^2 + O(5^3)             O(5^3)]
verbose 1 (add) [5^3 + O(5^4) 5^2 + O(5^3)       O(5^4)]
verbose 1 (add) [  4*5 + 4*5^2 + O(5^3) 4 + 5 + 3*5^2 + O(5^3)                 O(5^3)]
verbose 1 (add) [4*5 + 3*5^2 + O(5^3)           4 + O(5^3)               O(5^3)]
verbose 1 (add) [      5 + 5^2 + O(5^3) 1 + 4*5 + 5^2 + O(5^3)                 O(5^3)]
verbose 1 (add) [    2*5 + 5^2 + O(5^3) 2 + 2*5 + 5^2 + O(5^3)                 O(5^3)]
verbose 1 (add) [      4*5^2 + O(5^3) 4*5 + 4*5^2 + O(5^3)               O(5^3)]
verbose 1 (add) [    3*5 + 4*5^2 + O(5^3) 3 + 3*5 + 4*5^2 + O(5^3)                   O(5^3)]
verb

In [40]:
mul(5,P)

(4*5^2 + O(5^3) : 1 + O(5^3) : O(5^3))

In [41]:
mul(25,P)

(O(5^3) : 1 + O(5^3) : O(5^3))

In [42]:
mul(25,Oe)

(O(5^3) : 1 + O(5^3) : O(5^3))

In [43]:
special_weil(mul(3,P), mul(2,Oe), 25)

verbose 1 (special_weil) (O(5^3) : O(5^3) : 1 + O(5^3))
verbose 1 (addition_chain_to_mult) (1, 24, 25)
verbose 1 (addition_chain_to_mult) (12, 12, 24)
verbose 1 (addition_chain_to_mult) (6, 6, 12)
verbose 1 (addition_chain_to_mult) (3, 3, 6)
verbose 1 (addition_chain_to_mult) (1, 2, 3)
verbose 1 (addition_chain_to_mult) (1, 1, 2)
verbose 1 (special_weil) (1, 1, 8)
verbose 1 (special_weil) 1 + 4*5^2 + O(5^3)
verbose 1 (special_weil) 1 + 4*5^2 + O(5^3)
verbose 1 (special_weil) (1, 2, 8)
verbose 1 (special_weil) 1 + 4*5 + 5^2 + O(5^3)
verbose 1 (special_weil) 1 + 4*5 + O(5^3)
verbose 1 (special_weil) (3, 3, 4)
verbose 1 (special_weil) 1 + 4*5 + 5^2 + O(5^3)
verbose 1 (special_weil) 1 + 3*5 + 3*5^2 + O(5^3)
verbose 1 (special_weil) (6, 6, 2)
verbose 1 (special_weil) 1 + 5^2 + O(5^3)
verbose 1 (special_weil) 1 + 3*5 + 4*5^2 + O(5^3)
verbose 1 (special_weil) (12, 12, 1)
verbose 1 (special_weil) 1 + 4*5 + 2*5^2 + O(5^3)
verbose 1 (special_weil) 1 + 2*5 + 4*5^2 + O(5^3)
verbose 1 (special_weil

1 + 2*5^2 + O(5^3)

In [112]:
R.<a> = Zq(5^3,2, type="capped-abs")
E5= EllipticCurve(R, [3,0])


In [113]:
R

5-adic Unramified Extension Ring in a defined by x^3 + 3*x + 3

In [115]:
set_verbose(2)
add(E5.lift_x(3*a+3),E5.lift_x(3*a+3))

verbose 1 (add) [                                          O(5^2)                                           O(5^2)                                           O(5^2)]
[(2*a^2 + 3*a + 3) + (3*a^2 + 2*a + 2)*5 + O(5^2)   (4*a^2 + 2*a + 4) + (a^2 + 3*a + 3)*5 + O(5^2)                       a + (4*a^2 + a)*5 + O(5^2)]
[                                          O(5^2)                                           O(5^2)                                           O(5^2)]
verbose 1 (add) [        (4*a^2 + 1) + (4*a^2 + 2)*5 + O(5^2)     (a^2 + 2*a + 2) + (4*a^2 + 2)*5 + O(5^2) (2*a^2 + a + 4) + (a^2 + 3*a + 2)*5 + O(5^2)]
verbose 1 (add) ('asd', [        (4*a^2 + 1) + (4*a^2 + 2)*5 + O(5^2)     (a^2 + 2*a + 2) + (4*a^2 + 2)*5 + O(5^2) (2*a^2 + a + 4) + (a^2 + 3*a + 2)*5 + O(5^2)])
verbose 1 (add) [    (2*a + 3) + (3*a^2 + 1)*5 + O(5^2)       3*a^2 + (2*a^2 + 2*a)*5 + O(5^2) (a + 2) + (2*a^2 + 3*a + 1)*5 + O(5^2)]
verbose 1 (add) ('asd', [    (2*a + 3) + (3*a^2 + 1)*5 + O(5^2)       3*a^2 + (2*a^2 + 

verbose 1 (add) ('asd', [(2*a^2 + 3*a) + (3*a + 4)*5 + O(5^2)     (a + 1) + (3*a^2 + 1)*5 + O(5^2)                 (a^2 + 2*a) + O(5^2)])
verbose 1 (add) [        (2*a^2 + 4*a + 4) + (a + 4)*5 + O(5^2) (4*a^2 + a + 1) + (4*a^2 + 2*a + 3)*5 + O(5^2)         (a^2 + 1) + (a^2 + 2*a + 2)*5 + O(5^2)]
verbose 1 (add) ('asd', [        (2*a^2 + 4*a + 4) + (a + 4)*5 + O(5^2) (4*a^2 + a + 1) + (4*a^2 + 2*a + 3)*5 + O(5^2)         (a^2 + 1) + (a^2 + 2*a + 2)*5 + O(5^2)])
verbose 1 (add) [                    2 + (2*a^2 + a)*5 + O(5^2) (a^2 + 4*a + 2) + (3*a^2 + 4*a + 1)*5 + O(5^2)             (a^2 + a) + (3*a^2 + a)*5 + O(5^2)]
verbose 1 (add) ('asd', [                    2 + (2*a^2 + a)*5 + O(5^2) (a^2 + 4*a + 2) + (3*a^2 + 4*a + 1)*5 + O(5^2)             (a^2 + a) + (3*a^2 + a)*5 + O(5^2)])
verbose 1 (add) [        (4*a^2 + 3*a + 2) + (a + 4)*5 + O(5^2)               1 + (2*a^2 + 4*a + 3)*5 + O(5^2) (4*a^2 + 2*a + 2) + (4*a^2 + a + 1)*5 + O(5^2)]
verbose 1 (add) ('asd', [        (4*a^2 + 3*a + 2

verbose 1 (add) [(3*a^2 + a + 2) + (a^2 + 3*a + 3)*5 + O(5^2)           (4*a^2 + a) + (a^2 + 4)*5 + O(5^2)     (2*a^2 + 3*a + 4) + (a^2 + a)*5 + O(5^2)]
verbose 1 (add) ('asd', [(3*a^2 + a + 2) + (a^2 + 3*a + 3)*5 + O(5^2)           (4*a^2 + a) + (a^2 + 4)*5 + O(5^2)     (2*a^2 + 3*a + 4) + (a^2 + a)*5 + O(5^2)])
verbose 1 (add) [(a^2 + 2*a + 4) + (a^2 + 3*a + 1)*5 + O(5^2) (3*a^2 + 2*a) + (3*a^2 + 4*a + 2)*5 + O(5^2)   (4*a^2 + a + 3) + (a^2 + a + 1)*5 + O(5^2)]
verbose 1 (add) ('asd', [(a^2 + 2*a + 4) + (a^2 + 3*a + 1)*5 + O(5^2) (3*a^2 + 2*a) + (3*a^2 + 4*a + 2)*5 + O(5^2)   (4*a^2 + a + 3) + (a^2 + a + 1)*5 + O(5^2)])
verbose 1 (add) [                  (2*a^2 + a + 1) + 5 + O(5^2)   (4*a^2 + 4*a) + (3*a^2 + 3*a + 3)*5 + O(5^2) (3*a^2 + 3*a + 3) + (2*a^2 + a + 2)*5 + O(5^2)]
verbose 1 (add) ('asd', [                  (2*a^2 + a + 1) + 5 + O(5^2)   (4*a^2 + 4*a) + (3*a^2 + 3*a + 3)*5 + O(5^2) (3*a^2 + 3*a + 3) + (2*a^2 + a + 2)*5 + O(5^2)])
verbose 1 (add) [(3*a^2 + 3*a + 1) + (2*a^2

verbose 1 (add) [(a^2 + a + 4) + (4*a^2 + 3*a + 2)*5 + O(5^2)             (a^2 + 4) + (a^2 + a)*5 + O(5^2)     (a^2 + 2) + (2*a^2 + 4*a + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [(a^2 + a + 4) + (4*a^2 + 3*a + 2)*5 + O(5^2)             (a^2 + 4) + (a^2 + a)*5 + O(5^2)     (a^2 + 2) + (2*a^2 + 4*a + 4)*5 + O(5^2)])
verbose 1 (add) [        1 + (4*a^2 + a + 4)*5 + O(5^2) (3*a^2 + 2*a + 1) + (a + 2)*5 + O(5^2)     (3*a^2 + 3*a) + (a + 1)*5 + O(5^2)]
verbose 1 (add) ('asd', [        1 + (4*a^2 + a + 4)*5 + O(5^2) (3*a^2 + 2*a + 1) + (a + 2)*5 + O(5^2)     (3*a^2 + 3*a) + (a + 1)*5 + O(5^2)])
verbose 1 (add) [(a^2 + a + 2) + (2*a^2 + 2*a + 3)*5 + O(5^2)             (a + 2) + (a^2 + 3*a)*5 + O(5^2)           (4*a + 2) + (2*a^2 + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [(a^2 + a + 2) + (2*a^2 + 2*a + 3)*5 + O(5^2)             (a + 2) + (a^2 + 3*a)*5 + O(5^2)           (4*a + 2) + (2*a^2 + 4)*5 + O(5^2)])
verbose 1 (add) [      (4*a^2 + 2) + (2*a^2 + 4*a + 4)*5 + O(5^2) (4*a^2 + 4*a + 3) + (2*a^

verbose 1 (add) [          2*a + (4*a^2 + a + 4)*5 + O(5^2) (4*a^2 + 4*a + 2) + (4*a^2 + 4)*5 + O(5^2)           (a^2 + 2*a + 2) + 3*a*5 + O(5^2)]
verbose 1 (add) ('asd', [          2*a + (4*a^2 + a + 4)*5 + O(5^2) (4*a^2 + 4*a + 2) + (4*a^2 + 4)*5 + O(5^2)           (a^2 + 2*a + 2) + 3*a*5 + O(5^2)])
verbose 1 (add) [(2*a + 3) + (a^2 + 4*a)*5 + O(5^2)   3*a^2 + (3*a^2 + 3*a)*5 + O(5^2)   (a + 2) + (4*a^2 + 3)*5 + O(5^2)]
verbose 1 (add) ('asd', [(2*a + 3) + (a^2 + 4*a)*5 + O(5^2)   3*a^2 + (3*a^2 + 3*a)*5 + O(5^2)   (a + 2) + (4*a^2 + 3)*5 + O(5^2)])
verbose 1 (add) [        (3*a + 3) + (2*a^2 + a + 2)*5 + O(5^2)         (2*a + 1) + (2*a^2 + a + 2)*5 + O(5^2) (3*a^2 + 2*a + 3) + (2*a^2 + a + 1)*5 + O(5^2)]
verbose 1 (add) ('asd', [        (3*a + 3) + (2*a^2 + a + 2)*5 + O(5^2)         (2*a + 1) + (2*a^2 + a + 2)*5 + O(5^2) (3*a^2 + 2*a + 3) + (2*a^2 + a + 1)*5 + O(5^2)])
verbose 1 (add) [      (3*a + 3) + (a^2 + 3*a)*5 + O(5^2)   (2*a + 1) + (3*a^2 + a + 3)*5 + O(5^2) (3*a^2 + 2*a + 3

...

verbose 1 (add) [            a^2 + (4*a^2 + 2*a + 2)*5 + O(5^2)         (2*a^2 + 4) + (3*a^2 + 3*a)*5 + O(5^2) (a^2 + 2*a + 1) + (2*a^2 + 4*a + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [            a^2 + (4*a^2 + 2*a + 2)*5 + O(5^2)         (2*a^2 + 4) + (3*a^2 + 3*a)*5 + O(5^2) (a^2 + 2*a + 1) + (2*a^2 + 4*a + 4)*5 + O(5^2)])
verbose 1 (add) [              (4*a^2 + 4*a) + a*5 + O(5^2) (a^2 + 3*a) + (4*a^2 + 3*a + 1)*5 + O(5^2)     (a^2 + 2*a + 3) + (4*a + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [              (4*a^2 + 4*a) + a*5 + O(5^2) (a^2 + 3*a) + (4*a^2 + 3*a + 1)*5 + O(5^2)     (a^2 + 2*a + 3) + (4*a + 4)*5 + O(5^2)])
verbose 1 (add) [  (a^2 + 3*a + 3) + (4*a^2 + 3*a + 3)*5 + O(5^2)     (2*a^2 + 2*a) + (3*a^2 + 2*a + 4)*5 + O(5^2) (4*a^2 + 4*a + 4) + (4*a^2 + 3*a + 3)*5 + O(5^2)]
verbose 1 (add) ('asd', [  (a^2 + 3*a + 3) + (4*a^2 + 3*a + 3)*5 + O(5^2)     (2*a^2 + 2*a) + (3*a^2 + 2*a + 4)*5 + O(5^2) (4*a^2 + 4*a + 4) + (4*a^2 + 3*a + 3)*5 + O(5^2)])
verbose 1 (add) [        3 + (2

verbose 1 (add) ('asd', [(4*a^2 + a + 4) + (2*a^2 + 3*a + 1)*5 + O(5^2)             (2*a^2 + 1) + (4*a + 4)*5 + O(5^2)             (4*a^2 + a) + (2*a + 2)*5 + O(5^2)])
verbose 1 (add) [      (2*a^2 + 3*a) + (a^2 + 4)*5 + O(5^2)     (a + 1) + (4*a^2 + 4*a + 2)*5 + O(5^2) (a^2 + 2*a) + (2*a^2 + 4*a + 3)*5 + O(5^2)]
verbose 1 (add) ('asd', [      (2*a^2 + 3*a) + (a^2 + 4)*5 + O(5^2)     (a + 1) + (4*a^2 + 4*a + 2)*5 + O(5^2) (a^2 + 2*a) + (2*a^2 + 4*a + 3)*5 + O(5^2)])
verbose 1 (add) [        (a + 3) + (3*a^2 + 3*a)*5 + O(5^2)   (a^2 + 3*a + 4) + (4*a^2 + 2)*5 + O(5^2) (2*a^2 + 1) + (2*a^2 + 2*a + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [        (a + 3) + (3*a^2 + 3*a)*5 + O(5^2)   (a^2 + 3*a + 4) + (4*a^2 + 2)*5 + O(5^2) (2*a^2 + 1) + (2*a^2 + 2*a + 4)*5 + O(5^2)])
verbose 1 (add) [(2*a^2 + a) + (2*a^2 + 4*a + 3)*5 + O(5^2) (a^2 + 2*a + 4) + (3*a^2 + 4*a)*5 + O(5^2)                   3 + (a^2 + 2)*5 + O(5^2)]
verbose 1 (add) ('asd', [(2*a^2 + a) + (2*a^2 + 4*a + 3)*5 + O(5^2) (a^2 + 2*a 

verbose 1 (add) ('asd', [      (a^2 + 4*a + 4) + (a + 2)*5 + O(5^2) (2*a^2 + a + 2) + (3*a^2 + 3*a)*5 + O(5^2)         3*a + (4*a^2 + 2*a + 4)*5 + O(5^2)])
verbose 1 (add) [(a^2 + 3*a + 2) + (a^2 + 2*a + 4)*5 + O(5^2)         (4*a^2 + 4) + (4*a^2 + a)*5 + O(5^2)         (a^2 + a + 4) + (4*a + 3)*5 + O(5^2)]
verbose 1 (add) ('asd', [(a^2 + 3*a + 2) + (a^2 + 2*a + 4)*5 + O(5^2)         (4*a^2 + 4) + (4*a^2 + a)*5 + O(5^2)         (a^2 + a + 4) + (4*a + 3)*5 + O(5^2)])
verbose 1 (add) [(2*a^2 + 2*a + 4) + (2*a + 3)*5 + O(5^2)     (2*a + 4) + (a^2 + a + 4)*5 + O(5^2)         (3*a + 4) + (2*a + 2)*5 + O(5^2)]
verbose 1 (add) ('asd', [(2*a^2 + 2*a + 4) + (2*a + 3)*5 + O(5^2)     (2*a + 4) + (a^2 + a + 4)*5 + O(5^2)         (3*a + 4) + (2*a + 2)*5 + O(5^2)])
verbose 1 (add) [    (a^2 + 3*a + 4) + (a^2 + a + 3)*5 + O(5^2)       (4*a + 1) + (2*a^2 + 4*a + 1)*5 + O(5^2) (2*a^2 + 2*a + 4) + (2*a^2 + a + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [    (a^2 + 3*a + 4) + (a^2 + a + 3)*5 + O(5^2)       (

verbose 1 (add) ('asd', [                2*a + (4*a^2 + a)*5 + O(5^2) (4*a^2 + 4*a + 2) + (2*a^2 + 2*a)*5 + O(5^2)     (a^2 + 2*a + 2) + (3*a^2 + a)*5 + O(5^2)])
verbose 1 (add) [(3*a^2 + a + 3) + (2*a^2 + 3)*5 + O(5^2) (2*a^2 + 3*a + 1) + (a^2 + 1)*5 + O(5^2)             (a + 4) + (a + 4)*5 + O(5^2)]
verbose 1 (add) ('asd', [(3*a^2 + a + 3) + (2*a^2 + 3)*5 + O(5^2) (2*a^2 + 3*a + 1) + (a^2 + 1)*5 + O(5^2)             (a + 4) + (a + 4)*5 + O(5^2)])
verbose 1 (add) [    (3*a^2 + 4*a + 3) + (2*a^2 + 2*a)*5 + O(5^2)         (3*a^2 + 4*a + 4) + (4*a + 2)*5 + O(5^2) (4*a^2 + 4*a + 2) + (3*a^2 + 2*a + 2)*5 + O(5^2)]
verbose 1 (add) ('asd', [    (3*a^2 + 4*a + 3) + (2*a^2 + 2*a)*5 + O(5^2)         (3*a^2 + 4*a + 4) + (4*a + 2)*5 + O(5^2) (4*a^2 + 4*a + 2) + (3*a^2 + 2*a + 2)*5 + O(5^2)])
verbose 1 (add) [(4*a^2 + 4*a + 3) + (2*a^2 + 2*a + 1)*5 + O(5^2)           (4*a + 3) + (4*a^2 + a + 4)*5 + O(5^2)               (a + 3) + (3*a^2 + 4*a)*5 + O(5^2)]
verbose 1 (add) ('asd', [(4*a^2 + 4*a + 3) 

KeyboardInterrupt: 

In [166]:
M = Matrix(R, 2,2, [5*a,a,5,1])

In [167]:
M

[a*5 + O(5^2)   a + O(5^2)]
[  5 + O(5^2)   1 + O(5^2)]

In [170]:
M._echelon_in_place("scaled_partial_pivoting_valuation")

verbose 1 (<module>) generic in-place Gauss elimination on 2 x 2 matrix using scaled_partial_pivoting_valuation algorithm


ValueError: negative valuation

In [172]:
M._echelon_form_PID??

In [163]:
M

[1 + O(5^2) 1 + O(5^2)]
[    O(5^2)     O(5^2)]

In [127]:
M.hermite_form()

TypeError: Generic echelon form only defined over integral domains

In [33]:
line(P, P, Q)

1

In [51]:
Zq??

In [34]:
print(add(Q,Q))

(4 : 4 : 1)


In [35]:
def weil_pairing(self, Q, n):
    r"""
    Compute the Weil pairing of self and `Q` using Miller's algorithm.

    INPUT:

    - ``Q`` -- a point on self.curve().

    - ``n`` -- an integer `n` such that `nP = nQ = (0:1:0)` where
      `P` = self.

    OUTPUT:

    An `n`'th root of unity in the base field self.curve().base_field()

    EXAMPLES::

        sage: F.<a>=GF(2^5)
        sage: E=EllipticCurve(F,[0,0,1,1,1])
        sage: P = E(a^4 + 1, a^3)
        sage: Fx.<b>=GF(2^(4*5))
        sage: Ex=EllipticCurve(Fx,[0,0,1,1,1])
        sage: phi=Hom(F,Fx)(F.gen().minpoly().roots(Fx)[0][0])
        sage: Px=Ex(phi(P.xy()[0]),phi(P.xy()[1]))
        sage: O = Ex(0)
        sage: Qx = Ex(b^19 + b^18 + b^16 + b^12 + b^10 + b^9 + b^8 + b^5 + b^3 + 1, b^18 + b^13 + b^10 + b^8 + b^5 + b^4 + b^3 + b)
        sage: Px.weil_pairing(Qx,41) == b^19 + b^15 + b^9 + b^8 + b^6 + b^4 + b^3 + b^2 + 1
        True
        sage: Px.weil_pairing(17*Px,41) == Fx(1)
        True
        sage: Px.weil_pairing(O,41) == Fx(1)
        True

    An error is raised if either point is not n-torsion::

        sage: Px.weil_pairing(O,40)
        Traceback (most recent call last):
        ...
        ValueError: points must both be n-torsion

    A larger example (see :trac:`4964`)::

        sage: P,Q = EllipticCurve(GF(19^4,'a'),[-1,0]).gens()
        sage: P.order(), Q.order()
        (360, 360)
        sage: z = P.weil_pairing(Q,360)
        sage: z.multiplicative_order()
        360

    An example over a number field::

        sage: P,Q = EllipticCurve('11a1').change_ring(CyclotomicField(5)).torsion_subgroup().gens()  # long time (10s)
        sage: P,Q = (P.element(), Q.element())  # long time
        sage: (P.order(),Q.order())  # long time
        (5, 5)
        sage: P.weil_pairing(Q,5)  # long time
        zeta5^2
        sage: Q.weil_pairing(P,5)  # long time
        zeta5^3

    ALGORITHM:

    Implemented using Proposition 8 in [Mil2004]_.  The value 1 is
    returned for linearly dependent input points.  This condition
    is caught via a DivisionByZeroError, since the use of a
    discrete logarithm test for linear dependence, is much too slow
    for large `n`.

    AUTHOR:

    - David Hansen (2009-01-25)
    """
    P = self
    E = P.scheme()

    if not Q.scheme() is E:
        raise ValueError("points must both be on the same curve")

    # Test if P, Q are both in E[n]
    #if not ((n*P).is_zero() and (n*Q).is_zero()):
    #    raise ValueError("points must both be n-torsion")

    one = E.base_ring().one()

    # Case where P = Q
    if P == Q:
        return one

    # Case where P = O or Q = O
    if P.is_zero() or Q.is_zero():
        return one

    # The non-trivial case P != Q

    # Note (2010-12-29): The following code block should not be
    # used.  It attempts to reduce the pairing calculation to order
    # d = gcd(|P|,|Q|) instead of order n, but this approach is
    # counterproductive, since calculating |P| and |Q| is much
    # slower than calculating the pairing [BGN05].
    #
    # [BGN05] D. Boneh, E. Goh, and K. Nissim, "Evaluating 2-DNF
    # Formulas on Ciphertexts", TCC 2005, LNCS 3378, pp. 325-341.
    #
    # Reduction to order d = gcd(|P|,|Q|); value is a d'th root of unity
    # try:
    #     nP = P.order()
    # except AttributeError:
    #     nP = generic.order_from_multiple(P,n,operation='+')
    # try:
    #     nQ = Q.order()
    # except AttributeError:
    #     nQ = generic.order_from_multiple(Q,n,operation='+')
    # d = arith.gcd(nP,nQ)
    # if d==1:
    #     return one
    #
    # P = (nP//d)*P # order d
    # Q = (nQ//d)*Q # order d
    # n = d

    try:
        print("a>>>")
        pq = miller(P, Q, n)
        print(pq,">>>")
        qp = miller(Q, P, n)
        print(qp, pq,">>>")
        return ((-1)**n.test_bit(0))*(pq/qp)
    except ZeroDivisionError:
        return one

In [36]:
R = add(Q, Q); print(R)
print(add(P, R))
miller(Q, add(P, R), p)

(4 : 4 : 1)
(1 : 3 : 1)


ZeroDivisionError: element 0 of quotient polynomial ring not invertible

In [ ]:
miller(Q,P,p)

In [ ]:
weil_pairing(P,add(P,Q),3)

In [ ]:
weil_pairing(Q,P,3)

In [ ]:
weil_pairing(neg(Ee(0,-2)), P, 3)

In [ ]:
neg(P)

In [ ]:
neg(Ee(0,-2))

In [ ]:
miller(P, Q, 3)


In [ ]:
R = Integers(5^2)

In [ ]:
E= EllipticCurve(R, [3,0])

In [ ]:
P = E(R(5), R(1), R(0))

In [ ]:
P = E(R)(5,1,0, check=False)

In [ ]:
add(P, P)
R(10)/R(6)

In [ ]:
R(10)/R(6)